In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import streamlit as st

In [2]:
data = pd.read_excel("cars.xls")

In [3]:
data.shape

(804, 12)

In [4]:
data.head(5)

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1


In [5]:
data["Make"].unique()

array(['Buick', 'Cadillac', 'Chevrolet', 'Pontiac', 'SAAB', 'Saturn'],
      dtype=object)

In [6]:
df = data.copy()

In [37]:
# df = df.drop(columns=["Leather","Sound","Cruise"])


In [7]:
df.shape

(804, 12)

In [8]:
X = df.drop("Price", axis=1)
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Price     804 non-null    float64
 1   Mileage   804 non-null    int64  
 2   Make      804 non-null    object 
 3   Model     804 non-null    object 
 4   Trim      804 non-null    object 
 5   Type      804 non-null    object 
 6   Cylinder  804 non-null    int64  
 7   Liter     804 non-null    float64
 8   Doors     804 non-null    int64  
 9   Cruise    804 non-null    int64  
 10  Sound     804 non-null    int64  
 11  Leather   804 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 75.5+ KB


- pop line -- one code encoing gibi dönüşümlerini standart yapmamızı sağlar

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Preprocessing pipeline
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), ["Doors", "Cylinder", "Mileage", "Liter"]),
        ("cat", OneHotEncoder(), ["Make", "Model", "Trim", "Type"])
    ]
)


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

my_model = LinearRegression()

pipe = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', my_model)
])

pipe.fit(X_train, y_train)



Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Doors', 'Cylinder',
                                                   'Mileage', 'Liter']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)


Mean Squared Error: 697393.1791110976
R2 Score: 0.9912072832405728


## Stremlit ile modeli yayma/deploy/ kullanıma sunma


In [18]:
import streamlit as st
import pandas as pd

#price tahmin fonksiyonu tanımla
def price(make,model,trim,mileage,car_type,cylinder,liter,doors,cruise,sound,leather):
    input_data=pd.DataFrame({'Make':[make],
                             'Model':[model],
                             'Trim':[trim],
                             'Mileage':[mileage],
                             'Type':[car_type],
                             'Cylinder':[cylinder],
                             'Liter':[liter],
                             'Doors':[doors],
                             'Cruise':[cruise],
                             'Sound':[sound],
                             'Leather':[leather]})
    prediction = pipe.predict(input_data)[0]
    return prediction

st.title("Car Price Prediction")
st.write("araba özelliklerini seçin")

make = st.selectbox("Marka",df['Make'].unique())
model = st.selectbox("Model",df[df['Make'] == make]['Model'].unique())
trim_options = df[(df['Make'] == make) & (df['Model'] == model)]['Trim'].unique()
Trim = st.selectbox("Trim", trim_options)
mileage = st.number_input('Kilometre', 100, 60000)
car_type = st.selectbox("Araç Tipi", df['Type'].unique())

cylinder = st.selectbox('Silindir Sayısı', df['Cylinder'].unique())

liter = st.number_input('Litre', 1, 10)

doors = st.selectbox('Kapı Sayısı', df['doors'].unique())

cruise = st.radio("Hız Kontrol",[True, False])

sound = st.radio("Ses Sistemi", [True, False])

leather = st.selectbox("Deri Döşeme",[True, False])

prediction = price(make, model, Trim, mileage, car_type, cylinder, liter, doors, cruise, sound, leather)

if st.button("Tahmin"):
    st.write(f"Tahmin edilen fiyat: {prediction}")

